## Tutorial / How to use

In this tutorial we create a (simplified) synthetic galaxy image from scratch, along with its associated segmentation map, and then run the statmorph code on it.

### Setting up

We import some Python packages first. If you are missing any of these, please see the the Installation section of the README.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve
import photutils
import time
import statmorph
%matplotlib inline

**Creating a model galaxy image**

We assume that the image size is 240x240 pixels, and that the "true" light distribution is described by a 2D Sersic profile with the following parameters:

In [ ]:
ny, nx = 240, 240
y, x = np.mgrid[0:ny, 0:nx]
sersic_model = models.Sersic2D(
    amplitude=1, r_eff=20, n=2.5, x_0=120.5, y_0=96.5,
    ellip=0.5, theta=-0.5)
image = sersic_model(x, y)
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))

**Convolving with a PSF**

In practice, every astronomical image is the convolution of a "true" image with a point spread function (PSF), which depends on the optics of the telescope, atmospheric conditions, etc. Here we assume that the PSF is a simple 2D Gaussian distribution:

In [ ]:
size = 20  # on each side from the center
sigma_psf = 2.0
y, x = np.mgrid[-size:size+1, -size:size+1]
psf = np.exp(-(x**2 + y**2)/(2.0*sigma_psf**2))
psf /= np.sum(psf)
plt.imshow(psf, origin='lower', cmap='gray')

Now we convolve the image with the PSF.

In [ ]:
image = convolve(image, psf)
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))

**Adding noise**

Here we add homogeneous Gaussian background noise, optimistically assuming that the signal-to-noise ratio (S/N) is 100 at the effective radius (where we had defined the Sérsic profile amplitude as 1.0). For simplicity, we do not consider Poisson noise associated with the source itself.

In [ ]:
np.random.seed(1)
snp = 100.0
image += (1.0 / snp) * np.random.standard_normal(size=(ny, nx))
plt.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))

**Gain and weight maps**

The code will ask for one of two input arguments: (1) a weight map, which is a 2D array (of the same size as the input image) representing one standard deviation at each pixel value, or (2) the gain, a scalar that can be multiplied by the science image to obtain the number of electron counts per pixel. The gain parameter is used internally by statmorph to calculate the weight map.

Here we assume, also somewhat optimistically, that there is an average of 10,000 electron counts/pixel at the effective radius (where we had defined the amplitude as 1.0), so that the gain is 10,000.

In [ ]:
gain = 10000.0

**Creating a segmentation map**

Besides the image itself and the weight map/gain, the only other *required* argument is the segmentation map, which labels the pixels belonging to different sources. It is usually generated by specialized tools such as SExtractor, but here we create it using photutils:

In [ ]:
threshold = photutils.detect_threshold(image, 1.5)
npixels = 5  # minimum number of connected pixels
segm = photutils.detect_sources(image, threshold, npixels)

Although statmorph is designed to process all the sources labeled by the segmentation map, in this example we only focus on the main (largest) source found in the image.

In [ ]:
# Keep only the largest segment
label = np.argmax(segm.areas) + 1
segmap = segm.data == label
plt.imshow(segmap, origin='lower', cmap='gray')

We regularize a bit the shape of the segmentation map:

In [ ]:
segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
segmap = np.int64(segmap_float > 0.5)
plt.imshow(segmap, origin='lower', cmap='gray')

### Running statmorph

**Measuring morphological parameters**

Now that we have all the required data, we are ready to measure the morphology of the source just created. Note that we include the PSF as a keyword argument. In principle, this results in more correct Sersic profile fits, although it can also make the code run slower, depending on the size of the PSF.

In [ ]:
start = time.time()
source_morphs = statmorph.source_morphology(
    image, segmap, gain=gain, psf=psf)
print('Time: %g s.' % (time.time() - start))

In general, `source_morphs` is a list of objects, each corresponding to a labeled source in the image. Here we focus on the first (and only) labeled source.

In [ ]:
morph = source_morphs[0]

Now we print some of the morphological properties just calculated:

In [ ]:
print('xc_centroid =', morph.xc_centroid)
print('yc_centroid =', morph.yc_centroid)
print('ellipticity_centroid =', morph.ellipticity_centroid)
print('elongation_centroid =', morph.elongation_centroid)
print('orientation_centroid =', morph.orientation_centroid)
print('xc_asymmetry =', morph.xc_asymmetry)
print('yc_asymmetry =', morph.yc_asymmetry)
print('ellipticity_asymmetry =', morph.ellipticity_asymmetry)
print('elongation_asymmetry =', morph.elongation_asymmetry)
print('orientation_asymmetry =', morph.orientation_asymmetry)
print('rpetro_circ =', morph.rpetro_circ)
print('rpetro_ellip =', morph.rpetro_ellip)
print('rhalf_circ =', morph.rhalf_circ)
print('rhalf_ellip =', morph.rhalf_ellip)
print('r20 =', morph.r20)
print('r80 =', morph.r80)
print('Gini =', morph.gini)
print('M20 =', morph.m20)
print('F(G, M20) =', morph.gini_m20_bulge)
print('S(G, M20) =', morph.gini_m20_merger)
print('sn_per_pixel =', morph.sn_per_pixel)
print('C =', morph.concentration)
print('A =', morph.asymmetry)
print('S =', morph.smoothness)
print('sersic_amplitude =', morph.sersic_amplitude)
print('sersic_rhalf =', morph.sersic_rhalf)
print('sersic_n =', morph.sersic_n)
print('sersic_xc =', morph.sersic_xc)
print('sersic_yc =', morph.sersic_yc)
print('sersic_ellip =', morph.sersic_ellip)
print('sersic_theta =', morph.sersic_theta)
print('sky_mean =', morph.sky_mean)
print('sky_median =', morph.sky_median)
print('sky_sigma =', morph.sky_sigma)
print('flag =', morph.flag)
print('flag_sersic =', morph.flag_sersic)

Note that the fitted Sersic profile is in pretty good agreement with the "true" Sersic profile that we originally defined (n=2.5, r_eff=20, etc.). However, such agreement tends to deteriorate somewhat at higher noise levels and larger Sersic indices (not to mention that real galaxies are not always well described by Sersic profiles).

Other morphological measurements that are more general and more robust to noise, which are also calculated by statmorph, include the Gini-M20 (Lotz et al. 2004), CAS (Conselice 2003) and MID (Freeman et al. 2013) statistics, as well as the outer asymmetry (Wen et al. 2014) and shape asymmetry (Pawlik et al. 2016).

Also note that statmorph returns two quality flags:

1. ``flag`` : indicates the quality the basic morphological measurements, taking one of the following values: 0 (good), 1 (suspect), 2 (bad), or 4 (catastrophic). More details can be found [here](https://statmorph.readthedocs.io/en/latest/description.html#output).

2. ``flag_sersic`` : indicates if there was a problem/warning during the Sersic profile fitting: values of 0 and 1 indicate good and bad fits, respectively.

In general, ``flag <= 1`` should always be enforced, while ``flag_sersic == 0`` should only be used when one is interested in Sersic fits (which might fail for merging galaxies and other "irregular" objects).

**Examining the fitted Sersic profile**

Finally, we can reconstruct the fitted Sersic profile and examine its residual. Here we used the ``ConvolvedSersic2D`` class defined in ``statmorph``.

In [ ]:
ny, nx = image.shape
y, x = np.mgrid[0:ny, 0:nx]
fitted_model = statmorph.ConvolvedSersic2D(
    amplitude=morph.sersic_amplitude,
    r_eff=morph.sersic_rhalf,
    n=morph.sersic_n,
    x_0=morph.sersic_xc,
    y_0=morph.sersic_yc,
    ellip=morph.sersic_ellip,
    theta=morph.sersic_theta)
fitted_model.set_psf(psf)  # required when using ConvolvedSersic2D
image_model = fitted_model(x, y)
bg_noise = (1.0 / snp) * np.random.standard_normal(size=(ny, nx))
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(131)
ax.imshow(image, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Original image')
ax = fig.add_subplot(132)
ax.imshow(image_model + bg_noise, cmap='gray', origin='lower',
           norm=simple_norm(image, stretch='log', log_a=10000))
ax.set_title('Fitted model')
ax = fig.add_subplot(133)
residual = image - image_model
ax.imshow(residual, cmap='gray', origin='lower',
           norm=simple_norm(residual, stretch='linear'))
ax.set_title('Residual')

**Examining other morphological diagnostics**

For convenience, we also provide a ``make_figure`` function that can be used to visualize some of the basic morphological measurements carried out by ``statmorph``. This creates a multi-panel figure analogous to Fig. 4 from Rodriguez-Gomez et al. (2019).

In [ ]:
from statmorph.utils.image_diagnostics import make_figure
fig = make_figure(morph)

In [ ]:
fig.savefig('tutorial.png', dpi=150)
plt.close(fig)